In [16]:
import pandas as pd
import os
import json
import datetime as dt

In [17]:
def read_json(filepath):
    f = open(filepath)
    data = json.load(f)
    return pd.DataFrame.from_dict(data)

In [11]:
team_df = read_json("./esports-data/teams.json")
mapping_df = read_json("./esports-data/mapping_data.json")
game_df = read_json("./games/LPL_A_343996.json")

In [12]:
start_time_str =  game_df[game_df["eventType"] == "game_info"]["eventTime"].values[0]
start_time_str

'2023-08-08T11:01:18.234Z'

In [13]:
start_time = dt.datetime.strptime(start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
print(start_time)

2023-08-08 11:01:18.234000


In [14]:
# real full game time is 23:35
# https://www.youtube.com/watch?v=37JuelCihvM

# game_info -> game_end
end_time_str =  game_df[game_df["eventType"] == "game_end"]["eventTime"].values[0]
end_time = dt.datetime.strptime(end_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")

print(end_time - start_time)

0:24:36.747000


In [15]:
# first stats_update to game_end

new_start_time_str =  game_df[game_df["eventType"] == "stats_update"]["eventTime"].values[0]
new_start_time = dt.datetime.strptime(new_start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
print(end_time - new_start_time)

0:24:25.564000


In [18]:
# calcuate timestamp @ 14 mins and 25 mins to create dataframes with sectioned data
game_df["dateTime"] = pd.to_datetime(game_df["eventTime"]).dt.tz_localize(None)
early_game_cutoff = start_time + dt.timedelta(minutes=14)
late_game_cutoff = start_time + dt.timedelta(minutes=25)
early_game_df = game_df[game_df["dateTime"] <= early_game_cutoff]
mid_game_df = game_df[(game_df["dateTime"] >= early_game_cutoff) & (game_df["dateTime"] <= late_game_cutoff)]
late_game_df = game_df[game_df["dateTime"] >= late_game_cutoff]

In [23]:
def split_by_time(dataframe, early_game = 14, late_game = 25):
    new_df = dataframe.copy()
    new_df["dateTime"] = pd.to_datetime(new_df["eventTime"]).dt.tz_localize(None)
    early_game_cutoff = start_time + dt.timedelta(minutes=14)
    late_game_cutoff = start_time + dt.timedelta(minutes=25)
    early_game_df = new_df[new_df["dateTime"] <= early_game_cutoff]
    mid_game_df = new_df[(new_df["dateTime"] >= early_game_cutoff) & (game_df["dateTime"] <= late_game_cutoff)]
    late_game_df = new_df[new_df["dateTime"] >= late_game_cutoff]
    return (early_game_df, mid_game_df, late_game_df)

In [27]:
game_splits = split_by_time(game_df)
game_splits[0]

,eventTime,eventType,platformGameId,participants,stageID,sequenceIndex,gameName,gameVersion,statsUpdateInterval,playbackID,...,teamID,lane,buildingType,turretTier,goldGain,itemBeforeUndo,actualStartTime,wallTime,winningTeam,dateTime
0,2023-08-08T11:01:18.234Z,game_info,LPL_A:343996,"[{'keystoneID': 8437, 'hashedIP': '3VoTUXLOxMy...",1,0,tj_lpl_edg_wbg_game3,13.13.518.1870,1000.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:01:18.234
1,2023-08-08T11:01:18.234Z,queued_epic_monster_info,LPL_A:343996,NaN,1,1,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:01:18.234
2,2023-08-08T11:01:18.234Z,queued_epic_monster_info,LPL_A:343996,NaN,1,2,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:01:18.234
3,2023-08-08T11:01:29.417Z,stats_update,LPL_A:343996,"[{'ability4CooldownRemaining': 0, 'magicPenetr...",1,14,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:01:29.417
4,2023-08-08T11:01:29.450Z,queued_dragon_info,LPL_A:343996,NaN,1,15,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:01:29.450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,2023-08-08T11:15:16.580Z,item_purchased,LPL_A:343996,NaN,1,2552,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:15:16.580
1347,2023-08-08T11:15:16.749Z,stats_update,LPL_A:343996,"[{'ability4CooldownRemaining': 0, 'magicPenetr...",1,2555,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:15:16.749
1348,2023-08-08T11:15:16.782Z,item_purchased,LPL_A:343996,NaN,1,2556,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:15:16.782
1349,2023-08-08T11:15:17.183Z,item_purchased,LPL_A:343996,NaN,1,2557,tj_lpl_edg_wbg_game3,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-08-08 11:15:17.183
